# Match Transitivity
As input, we use the symmetric dataset

Here, I programmed an attempt to enforce transitivity with 4 matches or 3.
 * If four matches, we check the tree matched lines and we add the missing matches
 * After, if three matches, we do the same 

Unfortunately there are tricky cases and we would need a kind of Hungarian algorithm to solve it.

Pierre Nugues

In [ ]:
import pandas as pd
import json
from tqdm import tqdm

In [ ]:
# pd.set_option('display.max_colwidth', None)

In [ ]:
"""df = pd.read_json(
    "hf://datasets/albinandersson/nf-headword-linked/data.jsonl", lines=True)
df.to_json('~/nf.json')
    """

In [ ]:
df = pd.read_json('nf_sym.json')

In [ ]:
df.head(10)

## Example

In [ ]:
df[df['headword'] == 'Aagaard']

Another example that does not go to solve

In [ ]:
{'E2_160182', 'E2_165518', 'E2_50422', 'E2_52287', 'E2_53948'}

In [ ]:
df[df['entry_id'] == 'E2_160182']

In [ ]:
df[df['entry_id'] == 'E4_30164']

In [ ]:
df[df['entry_id'] == 'E1_23954']

In [ ]:
df[df['entry_id'] == 'E3_8384']

In [ ]:
df[df['entry_id'] == 'E2_41636']

In [ ]:
df[df['entry_id'] == 'E2_160182']

We have the links:
 * E2_41636 <-> E1_23954
 * E2_41636 <-> E3_8384
 * E1_23954 <-> E4_30164
 * E1_23954 <-> E3_8384
 * E3_8384 <-> E4_30164 <-> E2_160182

## Four Matches

Attempt to write a program to establish transitivity from E2 --> E1, E3, E4

In [ ]:
df.loc[(df['edition'] == 'E2') &
       (df['E1_match']) &
       (df['E3_match']) &
       (df['E4_match'])]

In [ ]:
idx_1 = df.loc[(df['edition'] == 'E1') &
               (df['E2_match']) &
               (df['E3_match']) &
               (df['E4_match'])].index

In [ ]:
idx_2 = df.loc[(df['edition'] == 'E2') &
               (df['E1_match']) &
               (df['E3_match']) &
               (df['E4_match'])].index

In [ ]:
idx_3 = df.loc[(df['edition'] == 'E3') &
               (df['E1_match']) &
               (df['E2_match']) &
               (df['E4_match'])].index

In [ ]:
idx_4 = df.loc[(df['edition'] == 'E4') &
               (df['E1_match']) &
               (df['E2_match']) &
               (df['E3_match'])].index

In [ ]:
df.iloc[117486][['E1_match', 'E3_match', 'E4_match']]

In [ ]:
df[df['entry_id'] == 'E1_85631']

In [ ]:
def missing_matches(df, idx, ed):
    all_eds = {'E1', 'E2', 'E3', 'E4'}
    compl_eds = all_eds.difference({ed})
    matching_eds = [ed + '_match' for ed in compl_eds]
    matching_entries = df.iloc[idx][matching_eds].to_list()
    matching_pairs = list(zip(matching_eds, matching_entries))
    for m in matching_pairs:
        class_members = set(matching_pairs).difference({m})
        for class_member in class_members:
            target_idx = df[df['entry_id'] == m[1]].index
            if not df.loc[target_idx, class_member[0]].any():
                df.loc[target_idx, class_member[0]] = class_member[1]
    return

In [ ]:
missing_matches(df, 117486, 'E2')

In [ ]:
df[df['entry_id'] == 'E1_85631']

In [ ]:
for idx in tqdm(idx_1):
    missing_matches(df, idx, 'E1')

In [ ]:
for idx in tqdm(idx_2):
    missing_matches(df, idx, 'E2')

In [ ]:
for idx in tqdm(idx_3):
    missing_matches(df, idx, 'E3')

In [ ]:
for idx in tqdm(idx_4):
    missing_matches(df, idx, 'E4')

In [ ]:
df.loc[(df['edition'] == 'E1') &
       (df['E2_match']) &
       (df['E3_match']) &
       (df['E4_match'])]

In [ ]:
df.loc[(df['edition'] == 'E2') &
       (df['E1_match']) &
       (df['E3_match']) &
       (df['E4_match'])]

In [ ]:
df.loc[(df['edition'] == 'E3') &
       (df['E1_match']) &
       (df['E2_match']) &
       (df['E4_match'])]

In [ ]:
df.loc[(df['edition'] == 'E4') &
       (df['E1_match']) &
       (df['E2_match']) &
       (df['E3_match'])]

In [ ]:
l1_2 = df.loc[(df['edition'] == 'E1') &
       (df['E2_match']) &
       (df['E3_match']) &
       (df['E4_match'])]['E2_match'].tolist()
l1_2 = sorted(l1_2)
len(set(l1_2))

In [ ]:
l2 = df.loc[(df['edition'] == 'E2') &
       (df['E1_match']) &
       (df['E3_match']) &
       (df['E4_match'])]['entry_id'].tolist()
l2 = sorted(l2)
len(set(l2))

In [ ]:
set(l1_2) ^ set(l2)

In [ ]:
df[df['entry_id'] == 'E1_23954']

In [ ]:
df[df['entry_id'] == 'E2_160182']

In [ ]:
#df.to_json('nf_sym_trans.json')

First failed attempt to write the symmetry program

In [ ]:
"""
def set_symmetry_old(df, ed1, ed2):
    concord_m, discord_m, no_m = 0, 0, 0
    matching_ed1 = ed1 + '_match'
    matching_ed2 = ed2 + '_match'
    for i in tqdm(range(len(df))):
        if df.iloc[i]['edition'] != ed1:
            continue
        entry = df.iloc[i]['entry_id']
        matching_entry = df.iloc[i][matching_ed2]
        if matching_entry:  # We have a matching entry
            # print(i, df.iloc[i]['entry_id'], matching_entry)
            if df[(df['edition'] == ed2) &
                  (df['entry_id'] == matching_entry) &
                    (df[matching_ed1] == entry)].any().any():  # Case 1: We have a symmetry
                concord_m += 1
                # print(df[df['entry_id'] == matching_entry])
            elif df[(df['edition'] == ed2) &
                    (df['entry_id'] == matching_entry) &
                    (df[matching_ed1] == '')].any().any():  # Case 2: We have a unique relation
                no_m += 1
                df.loc[df[df['entry_id'] == matching_entry].index,
                       matching_ed1] = entry
                # print(i, df.iloc[i]['entry_id'], matching_entry)
                # print(df[df['entry_id'] == matching_entry].index)
                # break
            else:  # Case 3: We have two nonsymmetrical relations. We remove them.
                # This will create one-way relations.
                # We need to run again the loop (twp-pass procedure)
                discord_m += 1
                df.loc[df[df['entry_id'] == entry].index,
                       matching_ed2] = ''
                df.loc[df[df['entry_id'] == matching_entry].index,
                       matching_ed1] = ''
                # print(i, df.iloc[i]['entry_id'], matching_entry)
                # print(df[df['entry_id'] == matching_entry].index)
                # break
    return concord_m, discord_m, no_m
"""